In [1]:
import torch, sys, os
from torch.nn import Module, Linear, SiLU
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn import global_add_pool
sys.path.append(os.path.abspath('/Users/samharshe/Documents/Gerstein Lab/EGNN Pro/src/data/get'))
from data_get_utils import get_dataset
from model_utils import bessel_rbf

In [3]:
benzene = get_dataset(version='charizard', molecule='benzene')
aspirin = get_dataset(version='charizard', molecule='aspirin')
uracil = get_dataset(version='charizard', molecule='uracil')

/usr/local/Caskroom/miniconda/base/envs/GDL/lib/python3.11/site-packages/torch_geometric/data/dataset.py:239: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-processing technique, pass `force_reload=True` explicitly to reload the dataset.
  warnings.warn(


In [9]:
print(max(benzene[:].edge_vec_length))
print(max(aspirin[:].edge_vec_length))
print(max(uracil[:].edge_vec_length))

tensor(1.4170)
tensor(1.1886)
tensor(1.4415)


In [31]:
class PAINNMessage(MessagePassing):
   def __init__():
        super(aggr='sum').__init__()
        
        self.act = SiLU()
        
        self.linear_1 = Linear(128, 128)
        self.linear_2 = Linear(20, 384)
        self.linear_3 = Linear(128, 384)
        
    def forward(self, data):
        data = self.propagate(edge_index=data.edge_index , x=data, edge_vectors=data.edge_vectors, edge_vectors_length=data.edge_vectors_length, unit_edge_vectors=data.unit_edge_vectors])
        
        return data
        
    def message(self, x_j, edge_attr, edge_vectors, edge_vectors_length, unit_edge_vectors):
        s_j = x_j.s_j
        v_j = x_j.v_j
        
        edge_vec = data.edge_vec
        edge_vec_length = data.edge_vec_length
        unit_edge_vec = data.unit_edge_vec
        
        rbf = bessel_rbf(x=edge_vec_length, n=20, r_cut=1.4415)
        
        s_j = self.linear_1(s_j)
        s_j = self.act(s_j)
        phi = self.linear_3(s_j)
        
        edge_attr = edge_attr
        rbf = rbf(edge_attr) # IMPLEMENT
        W = self.linear_2(edge_attr)
        
        split = phi * W
        split_1 = split[:128]
        split_2 = split[128:256]
        split_3 = split[256:]
        
        split_3 = split_3.view(128, 1) * r_hat_ij
        
        v_j = split_1 * v_j
        v_j = v_j + split_3
        s_j = split_2
        
        return v_j, s_j

    def update(self, aggr_out, x):
        x.v_i = x.v_i + aggr_out[0]
        x.s_i = x.s_i + aggr_out[1]
        
        return x